In [8]:
import os
root = os.getcwd().rsplit('/',1)[0]
vocab_path = '{0:s}/etc/vocab.ttl'.format(root)
dict_path = '{0:s}/etc/dict.ttl'.format(root)
print(vocab_path)
print(dict_path)

/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/vocab.ttl
/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/dict.ttl


In [12]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib
#graph = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')
gvocab = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')

# now add the data dictionary:
#graph.parse(dict_path, format='turtle')
gdict = rdflib.ConjunctiveGraph().parse(dict_path, format='turtle')
graph = gvocab + gdict

print(graph.serialize(format='n3').decode('ascii'))
#print(graph.identifier)
#print(gvocab.identifier)
#print(gdict.identifier)



@prefix : <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#> .
@prefix adms: <http://www.w3.org/ns/adms#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix logset: <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:console_logfile a logset:LogSeries ;
    logset:logFormat :timeStampedLogFile ;
    logset:logFormatInfo "filepattern=console",
        "filepattern=console-<dateYYYYMMDD>",
        "part_word=1",
        "ts_word=0" .

:messages_logfile a logset:LogSeries ;
    logset:logFormat :timeStampedLogFile 

In [115]:
gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
#graph.parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
graph = (gvocab + gdict) + gindex
#print(graph.serialize(format='n3').decode('ascii'))
#for s,p,o in graph:
for s,p,o in gindex:    
    print ((s,p,o))
    

(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/LogSet'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b6'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/File'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b3'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/File'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b6'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/path'), rdflib.term.Literal('p0-20170906t151820/messages-20170906'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b3'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/isLogOf'), rdflib.term.URIRef('http://example.com/owl/hpc-logset/console_logfile'))
(rdflib.term.BNode('f8554545d9f414665be1ab0

In [116]:
query = '''SELECT ?id ?desc
WHERE {
?id a :LogSet .
?id dct:description ?desc .
}'''
#print(query[21])
for row in graph.query(query):
    print(row)


(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b1'), rdflib.term.Literal('a partial sample of cori log data for testing logset tools'))


In [119]:
query = '''SELECT ?id ?path
WHERE {
?id :isLogOf :console_logfile .
?id :path ?path .
} '''
for row in graph.query(query):
    print(row)

(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))


In [130]:
query = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* :DataSource . 
OPTIONAL { ?id :path ?path }
} '''
for row in graph.query(query):
    print(row)

(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b7'), rdflib.term.Literal('p0-20170906t151820/messages-20170907'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b6'), rdflib.term.Literal('p0-20170906t151820/messages-20170906'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('f8554545d9f414665be1ab08daa99f3e8b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))


In [16]:
# can i parse a remote graph at all?
g1 = rdflib.Graph().parse('https://www.w3.org/ns/dcat.ttl')
print (g1)
gg = rdflib.Graph().parse('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab.ttl')
#gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
##for s,p,o in gindex:    
##    print ((s,p,o))
#for n in gindex.namespaces():
#    print (n)
#print(dir(rdflib.term.URIRef))
#print(rdflib.term.URIRef.__class__)
#for uri in [n[1] for n in gindex.namespaces() if n[0] in ('logset','dict')]:
#    print(str(uri.n3()))
#    print(str(uri)[:-1])
#    g = rdflib.ConjunctiveGraph().parse(str(uri)[:-1])
#    print(g.serialize(format='n3').decode('ascii'))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']].


PluginException: No plugin registered for (text/plain, <class 'rdflib.parser.Parser'>)

In [143]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isLogOf dict:console_logfile .
?id logset:path ?path .
} '''
query2 = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* logset:DataSource . 
OPTIONAL { ?id logset:path ?path }
} '''
query3 = '''SELECT ?id ?type ?base
WHERE {
?id a ?type .
OPTIONAL { ?type rdfs:subClassOf ?base }
}'''
for row in gindex.query(query3):
    print(row)

(rdflib.term.BNode('f9081ed9aa3d544bd9a1a36efa6c41312b7'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#File'), None)
(rdflib.term.BNode('f9081ed9aa3d544bd9a1a36efa6c41312b6'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#File'), None)
(rdflib.term.BNode('f9081ed9aa3d544bd9a1a36efa6c41312b3'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#File'), None)
(rdflib.term.BNode('f9081ed9aa3d544bd9a1a36efa6c41312b2'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#File'), None)
(rdflib.term.BNode('f9081ed9aa3d544bd9a1a36efa6c41312b1'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#LogSet'), None)
